<h1>MassBalanceMachine XGBoost Model Training - Iceland Region Example</h1>
<p style='text-align: justify;'>
In this notebook, we will simulate the glacier surface mass balance for the Iceland region using a custom <a href='https://xgboost.readthedocs.io/en/stable/'>XGBoost</a> model. The XGBoost model is designed with a custom objective function that generates monthly predictions based on aggregated observational data. We will create an instance of <code>CustomXGBoostRegressor</code> and train it using this custom loss function on the stake data from Iceland, which we have prepared in earlier notebooks. If you haven't already, please review the <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data preprocessing</a> and <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>data processing WGMS</a> notebooks for more details.</p>
<p style='text-align: justify;'>
The workflow includes several key steps:
<ol>
    <li><strong>Data Loading and Preparation:</strong> A <code>Dataloader</code> object is created to handle the loading of data and the creation of a training and testing split. This object also manages the generation of data splits for cross-validation.</li>
    <li><strong>Cross-Validation and Model Training:</strong> Using Scikit-learn's cross-validation techniques, we explore different hyperparameters and train the model on the prepared data splits. This approach ensures a robust evaluation and helps in selecting suitable parameters.</li>
    <li><strong>Aggregated Predictions:</strong> After training, we will display the aggregated monthly predictions generated by the model to visualize and analyze the results.</li>
    <li><strong>Model Evaluation:</strong> Finally, the model's performance is evaluated on the test set, providing insights into its predictive accuracy for glacier mass balance.</li>
</ol></p>

In [ ]:
import pandas as pd
import massbalancemachine as mbm

In [ ]:
data = pd.read_csv('./example_data/iceland/files/iceland_monthly_dataset.csv')
display(data)

<h2>1. Create the Train and Test Dataset and the Data Splits for Cross Validation</h2>
<p style='text-align: justify;'>
First, we create a <code>DataLoader</code> object, which generates both training and testing datasets, as well as the data splits required for cross-validation. To conserve memory, the <code>set_train_test_split</code> method returns iterators containing indices for the training and testing datasets. These indices are then used to retrieve the corresponding data for training and testing. Next, the <code>get_cv_split</code> method provides a list indicating the number of folds needed for cross-validation.
</p>


In [ ]:
# Create a new DataLoader object with the monthly stake data measurements.
dataloader = mbm.DataLoader(data=data)
# Create a training and testing iterators. The parameters are optional. The default value of test_size is 0.3.
train_itr, test_itr = dataloader.set_train_test_split(test_size= 0.3, random_seed=42, shuffle=True)

# Get all indices of the training and testing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, test_indices = list(train_itr), list(test_itr)

# Get the features and targets of the training data for the indices as defined above, that will be used during the cross validation.
df_X_train = data.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Create the cross validation splits based on the training dataset. The default value for the number of splits is 5.
splits = dataloader.get_cv_split(n_splits=5)

<h2>2. Create a CustomXGBoostRegressor Model</h2>
<p style='text-align: justify;'>
Next, we define the parameter ranges for each XGBoost parameter. In the subsequent step, we use cross-validation to explore these parameter ranges and select the combination that yields the lowest loss. Additionally, we create a <code>CustomXGBoostRegressor</code> object.
</p>

In [ ]:
# For each of the XGBoost parameter, define the grid range
parameters = {
    'max_depth': [3, 4, 5, 6,],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 1]
}

In [ ]:
# Create a CustomXGBoostRegressor instance
custom_xgboost = mbm.models.CustomXGBoostRegressor(device="cpu")

<h2>3. Train the CustomXGBoostRegressor model</h2>

In [ ]:
# The default value of num_jobs=-1, meaning all cores will be utilised during fitting the model.
# custom_xgboost.gridsearch(parameters=parameters, splits=splits, features=df_X_train, targets=y_train, num_jobs=-1)

custom_xgboost.randomsearch(parameters=parameters, n_iter=20, splits=splits, features=df_X_train, targets=y_train, num_jobs=-1)

<h3>3.1 Show the Predictions per Fold</h3>